# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на вопросы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные для исследования в файле `data.csv`. О качестве данных ничего не известно. Поэтому перед тем, чтобы ответить на вопросы,  понадобится обзор данных.

Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Ответы на вопросы.

## Обзор данных

Составим первое представление о данных клиентов.

Основной инструмент аналитика - pandas. Импортируем эту библиотеку.

In [1]:
# импорт библиотеки pandas
import pandas as pd

Прочитаем файл data.csv из папки /datasets и сохраним его в переменной df:

In [2]:
# чтение файла с данными и сохранение в df
borrower_bank = pd.read_csv('/datasets/data.csv')

Выведем на экран первые 15 строк таблицы:

In [3]:
# получение первых 15 строк таблицы df
borrower_bank.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Методом <i>.info()</i> узнаем общую информацию о таблице:

In [4]:
# получение общей информации о данных в таблице df методом info()
borrower_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Методом <i>describe()</i> узнаем общую информацию о таблице, но несколько в ином виде:

In [5]:
# получение общей информации о данных в таблице df методом describe()
borrower_bank.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Итак, в таблице 12 столбцов. Типы данных в столбцах : `int64`, `float64` и `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — образование клиента
* `education_id` — идентификатор образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — доход в месяц
* `purpose` — цель получения кредита

Название колонок записано верно.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Заметны некорректные данные:
* отрицательные значения в столбце со стажем,
* разная форма записей в столбце со стажем (если среднее число дней стажа перевести в года, то получится 172, это невозможно),
* в столбце количество детей тоже закрались некорректные записи (-1 и 20),
* минимальное значени возраста - 0 (это говорит о том, что данные были пропущены),

**Выводы**



В каждой строке таблицы - данные о клиенте, оформившего кредит в банке.

Предварительно можно утверждать, что данных достаточно для наших целей. Но встречаются пропуски в данных, а значения в некоторых колонках - некоректно записаны данные.

Чтобы ответить на вопросы, нужно устранить проблемы в данных.

##  Предобработка данных

### Обработка некоректных данных

Первым делом займемся столбцом `days_employed`:
* Воспользуемся функцией <i>abs</i> и методом <i> apply()</i> 

In [6]:
# присвоим значениям столбца 'days_employed' значения-результаты работы функции abs
borrower_bank['days_employed'] = borrower_bank['days_employed'].apply(abs)

* Переведем некоректные записи стажа (часы в дни)

Предположим, что работать разрешено с 16 лет, поэтому нам нужно найти значения, которые больше выражения: (Возраст - 16)* 365. Те значения, которые меньше или равны этому выражению записаны корректно, а остальные нет, поэтому мы поделим их на 24 (24 часа = 1 день).

In [7]:
# выполним логическую индексацию и проверим условие

for i in range(len(borrower_bank)):
    if borrower_bank.loc[i, 'days_employed'] > ((borrower_bank.loc[i, 'dob_years'] - 16)*365):
        borrower_bank.loc[i, 'days_employed'] = borrower_bank.loc[i, 'days_employed'] / 24

Выведем первые 10 строк и посмотрим на результат.

In [8]:
# прочитаем первые 10 строк таблицы df
borrower_bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Результат есть - стаж записан везде одинаково : в днях и без отрицательныз значений.

Теперь вернемся к столбцу `children`. Заметим что:
* есть отрицательное значение -1, что скорее всего ознает "нет детей", поэтому значение -1 поменяем на 0,
* значение 20 скорее всего не означает, что у клиента 20 детей, а то, что цифры 2 и 0 находят радом на клавиатуре, поэтому будем считать что 0 был дописан случайно и значения 20 поменяем на 2.

Для замены значений в столбце воспользуемся методом <i> replace() </i>:

In [9]:
# заменяем значения в столбце 'children' методом replace(), где первый аргумент - значение, которое нужно заменить (-1, 20),
# а второй - значение, на которое меняем (0, 2)

borrower_bank['children'] = borrower_bank['children'].replace({-1:0, 20:2})

### Обработка пропусков

Cначала посчитаем, сколько в таблице пропущенных значений. Для этого воспользуемся двумя методами `pandas`:

In [10]:
# подсчёт пропусков
borrower_bank.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим, что в столбцах `days_employed` и `total_income` есть пропущенные значения. Проверим:
* в одинаковых строках пропущенные значения или нет, 
* что представляют собой пропущенные значения в столбцах.

In [11]:
# при помощи логического И проверяем условие "в одинаковых строках пропущеные значения" 
# выводим информацию методом info()
borrower_bank[(borrower_bank['days_employed'].isna() == True) & (borrower_bank['total_income'].isna() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Предположение подтвердилось, теперь посмотрим какие это пропущенные значения.

In [12]:
# выводим 10 первых строк с пропущенными значениями
borrower_bank[borrower_bank['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пропущенными значениями в столбцах оказались NaN. Они замещают отсутсвующие в ячейке число и принадлежат к типу float.
Пропущенные значения неслучайные. Вероятность пропуска зависит от других значений, в том числе и от значений собственного столбца, т.е. клиент нарочно не указал свой стаж и доход.

Чтобы их изменить:
1. Создадим три Series без пропущенных значений, c группировкой DataFrame по столбцу `income_type` с данными столбцов стажа, возраста и дохода.
2. Посчитаем медиану для каждого столбца.
3. Заполним пропущенные значения значениями медианы в соответствии с должностью.
4. Возраст 0 - медианой по возрасту в соответствии с должностью.

In [13]:
# сгруппируем df без пропущенных значений по столбцу 'income_type'
# c выводом в столбце 'days_employed' медианного значения
df_days_employed = borrower_bank[borrower_bank.isna() == False].groupby('income_type')['days_employed'].median()
# выведем результат на экран
df_days_employed

income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2605.718536
компаньон           1523.272831
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1539.475707
студент              578.751554
Name: days_employed, dtype: float64

In [14]:
# сгрупируем df без пропущенных значений по столбцу 'income_type'
# c выводом в столбце 'total_income' медианного значения
df_total_income = borrower_bank[borrower_bank.isna() == False].groupby('income_type')['total_income'].median()
# выведем результат на экран
df_total_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [15]:
# сгруппируем df без строк, где возраст 0, по столбцу 'income_type'
# c выводом в столбце 'dob_years' медианного значения
df_dob_years = borrower_bank[borrower_bank['dob_years'] != 0].groupby('income_type')['dob_years'].median()
# выведем результат на экран
df_dob_years

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Данные получены, поэтому можно заполнять пропуски и менять возраст с 0 на соответсвующий, в зависимости от должности.

 - заполним пропуски столбцов `days_employed` и `total_income` 
 - заменим "нулевой" возраст в столбце `dob_years`

In [16]:
# импорт библиотеки numpy
import numpy as np

# циклом for перебираем условия (категории должностей), по которым перебираем следующие условия:
# строчки df, в которых встречается пропуск: если у обоих условий - True, 
# то перезаписываем NaN медианным значением из соотв. Series;
# значение возраста равно 0 - True, записываем значение из соотв. Series

for i in range(len(borrower_bank)):
    if (borrower_bank.loc[i, 'income_type'] == 'безработный'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[0]
            borrower_bank.loc[i, 'total_income'] = df_total_income[0]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[0]

    elif (borrower_bank.loc[i, 'income_type'] == 'в декрете'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[1]
            borrower_bank.loc[i, 'total_income'] = df_total_income[1]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[1]

    elif (borrower_bank.loc[i, 'income_type'] == 'госслужащий'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[2]
            borrower_bank.loc[i, 'total_income'] = df_total_income[2]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[2]

    elif (borrower_bank.loc[i, 'income_type'] == 'компаньон'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[3]
            borrower_bank.loc[i, 'total_income'] = df_total_income[3]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[3]

    elif (borrower_bank.loc[i, 'income_type'] == 'пенсионер'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[4]
            borrower_bank.loc[i, 'total_income'] = df_total_income[4]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[4]

    elif (borrower_bank.loc[i, 'income_type'] == 'предприниматель'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[5]
            borrower_bank.loc[i, 'total_income'] = df_total_income[5]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[5]

    elif (borrower_bank.loc[i, 'income_type'] == 'сотрудник'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[6]
            borrower_bank.loc[i, 'total_income'] = df_total_income[6]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[6]

    elif (borrower_bank.loc[i, 'income_type'] == 'студент'):
        if (np.isnan(borrower_bank.loc[i, 'days_employed'])):
            borrower_bank.loc[i, 'days_employed'] = df_days_employed[7]
            borrower_bank.loc[i, 'total_income'] = df_total_income[7]
        if (borrower_bank.loc[i, 'dob_years'] == 0):
            borrower_bank.loc[i, 'dob_years'] = df_dob_years[7]

In [17]:
# выведем первые 15 строк df
borrower_bank.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


При первом выводе данных в столбцах `days_employed` и `total_income` было NaN ( индекс 12), сейчас это наши полученные медианные значения, значит все верно.

Проверим минимальный возраст в столбце `dob_years` (ранее это было значение 0).

In [18]:
# выведем минимальный возраст
borrower_bank['dob_years'].min()

19.0

Минимальный возраст 19 лет, а это значит, что все некорреткные данные по возрасту изменились.

Убедимся, что в таблице не осталось пропущенных значений. Для этого еще раз поcчитаем пропуски.

In [19]:
# подсчёт пропусков
borrower_bank.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Некорректные данные стали корректными, пропуски заполнены медианными значениями по должностям, а это значит, что можно переходить к следующему этапу.

### Замена типа данных

Изменим тип данных столбцов `days_employed`,`total_income`, `dob_years` на int c помощью цикла for.

In [20]:
# запишем значения колонок в список columns_replace_type
# и методом astype() изменим тип данных на int
columns_replace_type = ['days_employed', 'total_income', 'dob_years' ]

borrower_bank[columns_replace_type] = borrower_bank[columns_replace_type].astype('int')

Проверим информацию о таблице после проделанных изменений методом <i>.info()</i>.

In [21]:
# получение общей информации о данных в таблице df
borrower_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные в таблице приняли визуально понятный вид. Самое время продолжить исследовать данные.

### Обработка дубликатов

Для того, чтобы начать искать явные дубликаты, важно заметить, что в столбцах `education` и `family_status` встречается разный регистр в значениях, что тоже может повлиять на нахождение явных дубликатов.

Поэтому для начала приведем значения в столбце `education` и `family_status` к нижнему регистру методом <i> str.lower()</i>.

In [22]:
# приведем значения в столбцах 'education' и 'family_status' к нижнему регистру методом str.lower()
borrower_bank['education'] = borrower_bank['education'].str.lower()
borrower_bank['family_status'] = borrower_bank ['family_status'].str.lower()

Теперь посчитаем явные дубликаты в таблице одной командой:

In [23]:
# подсчёт явных дубликатов
borrower_bank.duplicated().sum()

71

Вызовем специальный метод `pandas`, чтобы удалить явные дубликаты:

In [24]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
borrower_bank = borrower_bank.drop_duplicates().reset_index(drop=True)

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [25]:
# подсчёт явных дубликатов
borrower_bank.duplicated().sum()

0

Явные дубликаты удалены, и наша таблица стала еще более информативной. 
Причиной появления дубликатов мог послужить человеческий фактор. 

### Лемматизация

Теперь избавимся от неявных дубликатов в колонке `purpose`. Например, одна и та же цель может быть записана немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных целей, отсортированный в алфавитном порядке. Для этого:
* извлечём нужный столбец датафрейма, 
* применим к нему метод сортировки,
* для отсортированного столбца вызовем метод, который вернёт уникальные значения из столбца,
* результат запишем в переменную `purpose_list` и выведем его на экран.

In [26]:
# Просмотр уникальных целей 
purpose_list = borrower_bank['purpose'].sort_values().unique()
purpose_list 

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

В списке целей много повторяющихся категорий - неявных дубликатов. Сделаем лемматизацию списка `purpose_list`.

Импортируем библиотеку `pymystem3`.

In [27]:
# импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem() 

Создаем пустой список lemmas для записи списка лемм из получившего списка уникальных целей.

In [28]:
# создаем пустой список lemmas
lemmas = []

# с помощью цикла for лемматизируем каждую строчку списка purpose_list и записываем в список lemmas

for line in purpose_list:
    lemma = m.lemmatize(line)
    lemmas.append(lemma)

Результат выведем на экран:

In [29]:
print(lemmas)

[['автомобиль', '\n'], ['автомобиль', '\n'], ['высокий', ' ', 'образование', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['жилье', '\n'], ['заниматься', ' ', 'высокий', ' ', 'образование', '\n'], ['заниматься', ' ', 'образование', '\n'], ['на', ' ', 'покупка', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['недвижимость', '\n'], ['образование', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['операция', ' ', 'с', ' ', 'недвижимость', '\n'], ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилье', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'нед

Лемматизация прошла успешно и уже можно заметить, что некоторые цели совпадают по смыслу. Самое время распределить их по категорияем.

### Категоризация данных

Пришло время привести цели в порядок. Для этого разобьем по категориям полученные леммы:
* автомобиль,
* жилье или недвижимость,
* образование,
* свадьба.

Создадим функцию `purpose_category()`, с помощью которой разобьем на категории цели.

In [30]:
# Объявляется функция purpose_category() с параметром purpose.
# В переменной lemmas_row лематизируем строки purpose
# С помощью цикла for пройдемся по строкам и при совпадении какой-либо подстроки (заданные условия с помощью if)
# функция возвращает категорию цели, к которой относится подкатегория(строка purpose)

def purpose_category(purpose):
    lemmas_row = m.lemmatize(purpose)
    for row in lemmas_row:
        if 'авто' in row:
            return 'автомобиль'
        if ('недвиж' in row) or ('жил' in row):
            return  'недвижимость'
        if 'свад' in row:
            return  'свадьба'
        if 'образов' in row:
            return 'образование'

Перезапишем столбец в DataFrame  `purpose`, присвоив значения-результаты функции `purpose_category()` методом <i> apply() </i>.

In [31]:
# перезапишем столбец 'purpose' в df методом apply() 
borrower_bank['purpose'] = borrower_bank['purpose'].apply(purpose_category)

Категоризируем данные для столбца `children`  используя правила:
* меньше 1 ребенка - нет детей
* меньше или равно 2 детям и не равно 0 - один или два ребенка
* больше 2 детей - многодетные

In [32]:
# Объявляется функция children_category() с параметром children.
# функция возвращает категорию детей, используя правила

def сhildren_category(children):
    if children < 1:
        return 'нет детей'
    if (children > 0) and (children < 3):
        return  'один или два ребенка'
    if children > 2:
        return  'многодетные'   

Перезапишем столбец `children` в DataFrame , присвоив значения-результаты функции `children_category()` методом <i> apply() </i>.

In [33]:
# перезапишем столбец 'children' в df методом apply() 
borrower_bank['children'] = borrower_bank['children'].apply(сhildren_category)

Категоризируем данные столбца `total_income`. Для начала найдем минимальное, медианное и максимальное значение дохода и выведем на экран.

In [34]:
# подсчет медианы, минимального и максимального значений столбца 'total_income'
mediana = borrower_bank['total_income'].median()
minimum = borrower_bank['total_income'].min()
maximum = borrower_bank['total_income'].max()

# вывод на экран результатов
print('Минимальный доход: ', minimum)
print('Средний доход: ', mediana)
print('Максимальный доход: ', maximum)

Минимальный доход:  20667
Средний доход:  142594.0
Максимальный доход:  2265604


Чтобы категоризировать эти данные, посчитаем два средних значения : `mean_1` - среднее от минимального и среднего дохода, `mean_2` - среднее от среднего и высокого дохода. Эти медианны послежат чертой для деления дохода на группы (низкий доход, средний доход, высокий доход).

In [35]:
# запишем в переменные mean_1 и mean_2 список нужных значений
mean_1 = [minimum, mediana] 
mean_2 = [mediana, maximum] 

# произведем расчет средних значений - черту деления
mean_1 = sum(mean_1)/len(mean_1)
mean_2 = sum(mean_2)/len(mean_2)

Пришло время категоризовать столбец, все данные у нас есть.
* Низкий доход: меньше или равно `mean_1`
* Средний доход: больше `mean_1` и меньше или равно `mean_2`
* Высокий доход: больше `mean_2`

In [36]:
# Объявляется функция income_category() с параметром income.
# функция возвращает категорию доходов, используя правила

def income_category(income):
    if income <= mean_1:
        return 'низкий доход'
    if (income > mean_1) and (income <= mean_2):
        return  'средний доход'
    if income > mean_2:
        return  'высокий доход' 

Перезапишем столбец `total_income` в DataFrame , присвоив значения-результаты функции `income_category()` методом <i> apply() </i>.

In [37]:
# перезапишем столбец 'total_income' в df методом apply() 
borrower_bank['total_income'] = borrower_bank['total_income'].apply(income_category)

Посмотрим первые 5 строк нового DataFrame.

In [38]:
borrower_bank.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,один или два ребенка,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,средний доход,недвижимость
1,один или два ребенка,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,средний доход,автомобиль
2,нет детей,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,средний доход,недвижимость
3,многодетные,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,средний доход,образование
4,нет детей,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,средний доход,свадьба


С помощью категоризации данных мы категоризировали столбцы `purpose`, `children`, `income_type`. 
В дальнейшем, это поможет ответить на вопросы, ради которых и проводилось данное исследование.

**Выводы**

Предобработка обнаружила проблемы в данных:

- пропущенные значения,
- некорректные данный,
- дубликаты — явные и неявные.

Мы предположили образование некоректных данных, исправили их, исправили тип данных в столбцах `days_employed`, `total_income` и `dob_years`, чтобы упростить работу с таблицей.
В столбцах `education`, `family_status` значения привели к нижнему регистра. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на медиану.
Категоризировали значения из столбца `purpose` c помощью лемматизации. Так же без категоризации не обошлись и столбцы `children`, `income_type`. Результаты всех изменений перезаписали в соответсвующий столбец.

Теперь можно перейти к ответам на вопросы. 

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Убедимся, что в столбце `debt` уникальные значения : 0 и 1.

In [39]:
# найдем уникальные значения столбца 'debt'
borrower_bank['debt'].unique()

array([0, 1])

Значение 1 означает, что задолжности по кредиту есть, а 0 - что нет.
Сгруппируем df по столбцу `children`, и применим к столбцу `debt` две операции : найдем сумму и общее количество.

In [40]:
# сгруппироуем df по столбцу 'children' и по столбцу 'debt' вычислим сумму и количество методом agg()
borrower_bank.groupby('children').agg({'debt': ['sum', 'count']})

debt       
                       sum  count
children                         
многодетные             31    380
нет детей             1064  14138
один или два ребенка   646   6936

Создадим таблицу `df_chidren` со столбцами :
* `debtors` - клиенты, которые имеют задолженности банку,
* `clients` - общее число клиентов,
* `ratio`- процент должников от всех клиентов.

In [41]:
# создание столбцов, которые получили при группировке по столбцу 'children'
data = {'debtors': [31, 1064, 646], 'clients': [380, 14138, 6936]}

# создадим DataFrame df_chidren
df_chidren = pd.DataFrame(data, index = ['многодетные', 'нет детей', 'один или два ребенка'])

# добавим к df_chidren столбец 'ratio'
df_chidren['ratio'] = round((df_chidren['debtors'] / df_chidren['clients']),3)*100

#вывод df_chidren
df_chidren

,debtors,clients,ratio
многодетные,31,380,8.2
нет детей,1064,14138,7.5
один или два ребенка,646,6936,9.3


In [42]:
df_example = borrower_bank.groupby('children')['debt'].agg(['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
многодетные,380,31,8.16%,91.84%
нет детей,14138,1064,7.53%,92.47%
один или два ребенка,6936,646,9.31%,90.69%


**Вывод**

Из созданной нами таблицы видно, что клиенты, которые имеют один или два ребенка, более склонны не оплачивть кредит в срок, по сравнению с многодетными или кто не имеет детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгрупируем `borrower_bank` по столбцу `family_status`.

In [43]:
# сгруппироуем df по столбцу 'family_status' и по столбцу 'debt' вычислим сумму и количество методом agg()
borrower_bank.groupby(['family_status']).agg({'debt': ['sum', 'count']})

debt       
                       sum  count
family_status                    
в разводе               85   1195
вдовец / вдова          63    959
гражданский брак       388   4151
женат / замужем        931  12339
не женат / не замужем  274   2810

Создадим таблицу `df_family` со столбцами :
* `debtors` - клиенты, которые имеют задолженности банку,
* `clients` - общее число клиентов,
* `ratio`- процент должников от всех клиентов.

In [44]:
# создание столбцов, которые получили при группировке по столбцу 'family_status'
data = {'debtors': [85, 63, 388, 931, 274], 'clients': [1195, 959, 4151, 12339, 2810]}

# создадим DataFrame df_family
df_family = pd.DataFrame(data, index = ['в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем', 'не женат / не замужем'])

# добавим к df_family столбец 'ratio'
df_family['ratio'] = round((df_family['debtors'] / df_family['clients']),3)*100

#вывод df_family
df_family

,debtors,clients,ratio
в разводе,85,1195,7.1
вдовец / вдова,63,959,6.6
гражданский брак,388,4151,9.3
женат / замужем,931,12339,7.5
не женат / не замужем,274,2810,9.8


**Вывод**

Из созданной нами таблицы можно сделать вывод, что клиенты, которые взяли кредит в банке и находятся в гражданском браке или не замужем / не женаты больше склонны к просроченному платежучем другие.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем `borrower_bank` по столбцу `total_income`.

In [45]:
# сгруппироуем df по столбцу 'total_income' и по столбцу 'debt' вычислим сумму и количество методом agg()
borrower_bank.groupby(['total_income']).agg({'debt': ['sum', 'count']})

debt       
                sum  count
total_income              
высокий доход     1     13
низкий доход    188   2454
средний доход  1552  18987

Создадим таблицу `df_income` со столбцами :
* `debtors` - клиенты, которые имеют задолженности банку,
* `clients` - общее число клиентов,
* `ratio`- процент должников от всех клиентов.

In [46]:
# создание столбцов, которые получили при группировке по столбцу 'total_income'
data = {'debtors': [188, 1552, 1], 'clients': [2454, 18987, 13]}

# создадим DataFrame df_income
df_income = pd.DataFrame(data, index = ['низкий доход', 'средний доход', 'высокий доход'])

# добавим к df_income столбец 'ratio'
df_income['ratio'] = round((df_income['debtors'] / df_income['clients']),4)*100

#вывод df_income
df_income

,debtors,clients,ratio
низкий доход,188,2454,7.66
средний доход,1552,18987,8.17
высокий доход,1,13,7.69


**Вывод**

Как ни странно, но большая вероятность задолженности банку при взятии кредита у клиент ов со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

Сгруппируем `borrower_bank` по столбцу `purpose`.

In [47]:
# сгруппироуем df по столбцу 'purpose' и по столбцу 'debt' вычислим сумму и количество методом agg()
borrower_bank.groupby(['purpose']).agg({'debt': ['sum', 'count']})

debt       
              sum  count
purpose                 
автомобиль    403   4306
недвижимость  782  10811
образование   370   4013
свадьба       186   2324

Создадим таблицу `df_purpose` со столбцами :
* `debtors` - клиенты, которые имеют задолженности банку,
* `clients` - общее число клиентов,
* `ratio`- коээфициент заемщиков, которые имеют задолжности банку, измеряется в процентах.

In [48]:
# создание столбцов, которые получили при группировке по столбцу 'purpose'
data = {'debtors': [403, 782, 370, 186], 'clients': [4306, 10811, 4013, 2324]}

# создадим DataFrame df_purpose
df_purpose = pd.DataFrame(data, index = ['автомобиль', 'недвижимость', 'образование', 'свадьба'])

# добавим к df_purpose столбец 'ratio'
df_purpose['ratio'] = round((df_purpose['debtors'] / df_purpose['clients']),4)*100

#вывод df_purpose
df_purpose

,debtors,clients,ratio
автомобиль,403,4306,9.36
недвижимость,782,10811,7.23
образование,370,4013,9.22
свадьба,186,2324,8.00


**Вывод**

Клиенты, которые взяли кредит на автомобиль, чаще всего имеют задолженности по кредиту.

## Итоги исследования

*При первом открытии данных мы заметили некоторые некорректные данные и приступили к их предобработке*:

1. Обработали некорректные значения:
 - Отрицательные значения в столбце со стажем иразная форма записей: избавились от отрицательных значений и привели значения стажа к одному формату записи (в количествах дней).
 - В столбце количество детей тоже закрались некорректные записи (-1 и 20) : заменили значения "-1" на 0, а 20 на 2.
2. Обработали и заменили по возможности пропуски:
 - Заполнили пропущенные значения значениями медианы в соответствии с должностью.
 - Возраст 0 заменили медианой по возрасту в соответствии с должностью.
3. Удалили явные дубликаты
4. Произвели лематизацию в нужных столбцах и произвели категоризацию данных.

*Мы изучили данные и ответили на вопросы*:
 1. **Есть ли зависимость между наличием детей и возвратом кредита в срок?**
 
Заемщики, не имеющие детей, менее склонны к просрочке по выплатам кредита.

 2. **Есть ли зависимость между семейным положением и возвратом кредита в срок?**
 
Заемщики, которые находятся в разводе или потеряли мужа / жену, чаще всего не имеют задолженностей по кредиту.
 
 3. **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**
 
Люди, более с низким или высоким доходом чаще отдают кредит в срок, чем люди имеющие средний доход.
 
 4. **Как разные цели кредита влияют на его возврат в срок?** 
 
Клиенты, целью кредита которых является автомобиль или образование, более сколнны к просроченным платежам по кредиту.